In [1]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [3]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [5]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-09-14 11:33:03,033] A new study created in memory with name: no-name-2e481d2c-d91d-4918-85d9-f649a72efc1d
[I 2025-09-14 11:33:05,212] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 161, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-14 11:33:06,715] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 107, 'max_depth': 11}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-14 11:33:08,997] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 168, 'max_depth': 14}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-14 11:33:11,499] Trial 3 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 200, 'max_depth': 6}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-14 11:33:13,791] Trial 4 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 162, 'max_depth': 4}. Best is trial 2 with value: 0.772811

In [6]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7858472998137803
Best hyperparameters: {'n_estimators': 121, 'max_depth': 15}


In [7]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.76


## Samplers in Optuna

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [9]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-09-14 11:34:25,457] A new study created in memory with name: no-name-50f7b8bb-8372-403b-8c51-28162b04cf94
[I 2025-09-14 11:34:26,617] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 65, 'max_depth': 9}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-09-14 11:34:27,986] Trial 1 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 80, 'max_depth': 8}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-09-14 11:34:29,856] Trial 2 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 145, 'max_depth': 16}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-09-14 11:34:31,678] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 149, 'max_depth': 5}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-09-14 11:34:33,987] Trial 4 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 155, 'max_depth': 11}. Best is trial 1 with value: 0.770949720

In [10]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 116, 'max_depth': 19}


In [11]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [12]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [13]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-09-14 11:36:01,372] A new study created in memory with name: no-name-3070ae3c-2884-4397-ae31-a55d200dbfee
[I 2025-09-14 11:36:02,735] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-14 11:36:04,880] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-14 11:36:05,955] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-14 11:36:08,062] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-14 11:36:09,364] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [14]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [15]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [16]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [17]:
# 1. Optimization History
plot_optimization_history(study).show()

In [18]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [19]:
# 3. Slice Plot
plot_slice(study).show()

In [20]:
# 4. Contour Plot
plot_contour(study).show()

In [21]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [22]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [23]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [24]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-09-14 11:36:42,069] A new study created in memory with name: no-name-18536ff7-c775-42b5-aac0-090940d54a4d


[I 2025-09-14 11:36:43,750] Trial 0 finished with value: 0.7746741154562384 and parameters: {'classifier': 'RandomForest', 'n_estimators': 132, 'max_depth': 8, 'min_samples_split': 2, 'min_samples_leaf': 8, 'bootstrap': False}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-09-14 11:36:47,912] Trial 1 finished with value: 0.7672253258845437 and parameters: {'classifier': 'RandomForest', 'n_estimators': 268, 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 2, 'bootstrap': True}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-09-14 11:36:48,534] Trial 2 finished with value: 0.7858472998137801 and parameters: {'classifier': 'SVM', 'C': 78.63098109599632, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 2 with value: 0.7858472998137801.
[I 2025-09-14 11:36:51,789] Trial 3 finished with value: 0.7672253258845437 and parameters: {'classifier': 'RandomForest', 'n_estimators': 224, 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 4, 'bootstrap': Fal

In [25]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.13814238011145433, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [26]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.774674,2025-09-14 11:36:42.073874,2025-09-14 11:36:43.750576,0 days 00:00:01.676702,NaN,False,RandomForest,NaN,NaN,NaN,8.0,8.0,2.0,132.0,COMPLETE
1,1,0.767225,2025-09-14 11:36:43.752587,2025-09-14 11:36:47.912852,0 days 00:00:04.160265,NaN,True,RandomForest,NaN,NaN,NaN,20.0,2.0,7.0,268.0,COMPLETE
2,2,0.785847,2025-09-14 11:36:47.914599,2025-09-14 11:36:48.534855,0 days 00:00:00.620256,78.630981,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.767225,2025-09-14 11:36:48.536843,2025-09-14 11:36:51.788179,0 days 00:00:03.251336,NaN,False,RandomForest,NaN,NaN,NaN,6.0,4.0,3.0,224.0,COMPLETE
4,4,0.785847,2025-09-14 11:36:51.791186,2025-09-14 11:36:52.121841,0 days 00:00:00.330655,27.046379,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.709497,2025-09-14 11:38:18.429529,2025-09-14 11:38:18.521045,0 days 00:00:00.091516,0.101205,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.789572,2025-09-14 11:38:18.522981,2025-09-14 11:38:18.587645,0 days 00:00:00.064664,0.140885,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.785847,2025-09-14 11:38:18.592375,2025-09-14 11:38:18.663239,0 days 00:00:00.070864,0.301118,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.770950,2025-09-14 11:38:18.665228,2025-09-14 11:38:20.659142,0 days 00:00:01.993914,NaN,False,RandomForest,NaN,NaN,NaN,17.0,4.0,9.0,191.0,COMPLETE


In [27]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 79
RandomForest        11
GradientBoosting    10
Name: count, dtype: int64

In [28]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.748231
RandomForest        0.767564
SVM                 0.775876
Name: value, dtype: float64

In [29]:
# 1. Optimization History
plot_optimization_history(study).show()

In [30]:
# 3. Slice Plot
plot_slice(study).show()

In [31]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [32]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-09-14 11:38:22,726] A new study created in memory with name: no-name-ec36e1f2-75fe-447e-b7a6-952863dc0dee
[W 2025-09-14 11:38:22,746] Trial 0 failed with parameters: {'lambda': 0.0005457593571796749, 'alpha': 0.0019411879555760437, 'eta': 0.09880373222432907, 'gamma': 2.2380449814650623e-08, 'max_depth': 4, 'min_child_weight': 5, 'subsample': 0.9102986353049541, 'colsample_bytree': 0.5574000047489812} because of the following error: ModuleNotFoundError('\nCould not find `optuna-integration` for `xgboost`.\nPlease run `pip install optuna-integration[xgboost]`.').
Traceback (most recent call last):
  File "c:\Users\nasrullah\.conda\envs\python_ml\Lib\site-packages\optuna\integration\xgboost.py", line 5, in <module>
    from optuna_integration.xgboost import XGBoostPruningCallback
ModuleNotFoundError: No module named 'optuna_integration'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\nasrullah\.conda\envs\p

ModuleNotFoundError: 
Could not find `optuna-integration` for `xgboost`.
Please run `pip install optuna-integration[xgboost]`.

In [ ]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()